In [ ]:
%pip install -q google-colab-selenium
import google_colab_selenium as gs
driver = gs.Chrome()

!pip install selenium
!pip install webdriver-manager




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 30.9 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from datetime import datetime
import pytz


utc = pytz.utc
local_tz = pytz.timezone('Europe/Istanbul')
# making time adjustments
def convert_to_local(utc_time):
    try:

        utc_dt = datetime.strptime(utc_time, "%b %d, %Y %I:%M %p")

        local_dt = utc_dt.replace(tzinfo=utc).astimezone(local_tz)
        return local_dt.strftime("%b %d, %Y %I:%M %p")
    except Exception as e:
        print(f"Error converting timestamp: {e}")
        return "Error"

# required libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Create launch options for Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--ignore-certificate-errors")

# start webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

try:
    # url
    driver.get("https://discuss.huggingface.co/c/gradio/26/l/top")

    # scrolling 1 time
    scroll_attempts = 1
    for _ in range(scroll_attempts):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  #waiting time

    # waiting time
    time.sleep(5)

    # finding functions
    topics = driver.find_elements(By.CLASS_NAME, "title")
    owners = driver.find_elements(By.CLASS_NAME, "posters")
    replies_buttons = driver.find_elements(By.CLASS_NAME, "btn-link.posts-map.badge-posts")
    views = driver.find_elements(By.CLASS_NAME, "number")
    activities = driver.find_elements(By.CLASS_NAME, "age")

    topic_titles = [topic.text for topic in topics]
    topic_links = [topic.get_attribute("href") for topic in topics]
    topic_owners = []
    for owner in owners:
        try:

            owner_names = [img.get_attribute("title") for img in owner.find_elements(By.TAG_NAME, "img")]
            topic_owners.append(", ".join(owner_names))
        except:
            topic_owners.append("Error")


    topic_replies = []
    for button in replies_buttons:
        try:

            reply_count = button.get_attribute("title").split()[0]
            topic_replies.append(reply_count)
        except:
            topic_replies.append("0")
    topic_views = []
    for view in views:
        try:
            title = view.get_attribute("title")
            if title and "this topic has been viewed" in title:
                view_count = title.split(" ")[5]
                topic_views.append(view_count)
        except:
            topic_views.append("0")

    topic_activities = []
    for activity in activities:
        try:
            activity_info = activity.get_attribute("title")
            topic_activities.append(activity_info)
        except:
            topic_activities.append("0")

    timestamps = []
    for activity in activities:
        try:
            activity_info = activity.get_attribute("title")
            topic_activities.append(activity_info)

            if "Created:" in activity_info:
                created_index = activity_info.find("Created:")
                timestamp = activity_info[created_index + 9:].split("\n")[0].strip()
                local_time = convert_to_local(timestamp)
                timestamps.append(local_time)
            else:
                timestamps.append("0")
        except:
            topic_activities.append("0")
            timestamps.append("0")
    topic_contents = []
    detailed_users = []
    detailed_answers = []
    detailed_timestamps = []

    for link in topic_links:
      try:
          # Go to link
          driver.get(link)
          time.sleep(3)

          try:
              original_post = driver.find_element(By.CSS_SELECTOR, ".regular.contents .cooked").text
              topic_contents.append(original_post)
          except Exception as e:
              topic_contents.append("0")
              print(f"Error fetching original post for {link}: {e}")


          try:
              usernames = driver.find_elements(By.CSS_SELECTOR, "div.names a")
              user_list = [username.text for username in usernames]
              detailed_users.append(user_list)
          except Exception as e:
              detailed_users.append([])
              print(f"Error fetching usernames for {link}: {e}")


          try:
             answers = driver.find_elements(By.CSS_SELECTOR, "div.cooked")
             valid_answers = []
             for answer in answers:

                if answer.find_elements(By.TAG_NAME, "blockquote"):
                  continue
                valid_answers.append(answer.text.strip())
             detailed_answers.append(valid_answers)
          except Exception as e:
              detailed_answers.append([])
              print(f"Error fetching answers for {link}: {e}")

          try:
              timestamps_1 = driver.find_elements(By.CSS_SELECTOR, "div.post-info.post-date span[title]")
              valid_timestamps = []
              for timestamp_1 in timestamps_1:
                try:
                  timestamp_1_value = timestamp_1.get_attribute("title")
                  valid_timestamps.append(timestamp_1_value.strip())
                except Exception as e:
                  valid_timestamps.append("0")
                  print(f"Error fetching timestamp: {e}")
              detailed_timestamps.append(valid_timestamps)
          except Exception as e:
              detailed_timestamps.append([])
              print(f"Error fetching timestamps for {link}: {e}")

      except Exception as e:
          print(f"Error processing topic link {link}: {e}")
          topic_contents.append("0")
          detailed_users.append([])
          detailed_answers.append([])

    topic_replies = [" "] + topic_replies
    topic_views = [" "] + topic_views
    topic_activities = [" "] + topic_activities
    timestamps = [" "] + timestamps
    detailed_timestamps = [" "] + detailed_timestamps


    max_length = len(topic_titles)
    if len(topic_replies) > len(topic_titles):
      topic_replies = topic_replies[:len(topic_titles)]

    while len(topic_owners) < max_length:
      topic_owners.append("Error")
    while len(topic_replies) < max_length:
     topic_replies.append("Error")
    while len(topic_views) < max_length:
     topic_views.append("Error")
    while len(topic_activities) < max_length:
     topic_activities.append("Error")
    while len(timestamps) < max_length:
     timestamps.append("Error")

    while len(detailed_users) < len(topic_titles):
     detailed_users.append(["Error"])

    while len(detailed_answers) < len(topic_titles):
     detailed_answers.append(["Error"])

    while len(detailed_timestamps) < len(topic_titles):
      detailed_timestamps.append(["Error"])


    max_users = max(len(users) for users in detailed_users)
    max_answers = max(len(answers) for answers in detailed_answers)
    max_entries = max(max_users, max_answers)


    # Convert data to Data Frame
    df = pd.DataFrame({
        "Topic Title": topic_titles,
        "Topic Link": topic_links,
        "Topic Owner": topic_owners,
        "Replies": topic_replies[:max_length],
        "Views": topic_views[:max_length],
        "Activity": topic_activities[:max_length],
        "Timestamp": timestamps[:max_length],
        "Topic Content":topic_contents[:max_length]
    })
    for i in range(max_entries):
      df[f"username_topic_answer_{i+1}"] = [
        users[i] if i < len(users) else "Error" for users in detailed_users
    ]
      df[f"answer_topic_answer_{i+1}"] = [
        answers[i] if i < len(answers) else "Error" for answers in detailed_answers
    ]
      df[f"timestamp_topic_answer_{i+1}"] = [
        timestamps[i] if i < len(timestamps) else "Error" for timestamps in detailed_timestamps
    ]

    df = df.replace("Error", " ")  
    file_name = "yourfile.xlsx"  
    
    df.to_excel(file_name, index=False, engine='openpyxl') 

    print(f"Titles successfully saved to file.")
finally:
    # close browser
    driver.quit()

from google.colab import files
files.download(file_name)



Error processing topic link None: Message: invalid argument: 'url' must be a string
  (Session info: chrome=131.0.6778.204)
Stacktrace:
#0 0x58939a5c38fa <unknown>
#1 0x58939a0d4d20 <unknown>
#2 0x58939a167273 <unknown>
#3 0x58939a147af2 <unknown>
#4 0x58939a166560 <unknown>
#5 0x58939a147893 <unknown>
#6 0x58939a11630d <unknown>
#7 0x58939a11732e <unknown>
#8 0x58939a59000b <unknown>
#9 0x58939a593f97 <unknown>
#10 0x58939a57c71c <unknown>
#11 0x58939a594b17 <unknown>
#12 0x58939a5616cf <unknown>
#13 0x58939a5b26b8 <unknown>
#14 0x58939a5b2880 <unknown>
#15 0x58939a5c2776 <unknown>
#16 0x786aae7cdac3 <unknown>

Titles successfully saved to file.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>